In [9]:
import json
import csv


In [10]:
def writeMongo(row):
    print (row)
    
def conversion(old):
    #"40°29'21.84'' N"
    grados, minutos, segundos, direction = re.split(u'[°\'"\s]+',old)
    #print ('-'.join ([grados,minutos,segundos]))
    factor = {'N':1, 'S':-1, 'E': 1, 'W':-1}
    res= (int(grados)+int(minutos)/60.0+float(segundos)/3600.0) * factor[direction]
    return round(res,7)
latLon = conversion ("78°55'44.295'' N")
if 78.9289708 == latLon :
    print ("conversion ok")
else:
    print (latLon)


conversion ok


In [11]:
jsonCallejero = open ('data/Callejero.json','w')

with open ('data/CallejeroUtf8.csv') as fCallejero:
        reader = csv.DictReader(fCallejero, delimiter=';')
        street = {}
        for row in reader:
            name = row['VIA_CLASE']+' '+\
                   row['VIA_PAR']+' '+\
                   row['VIA_NOMBRE']
            if not 'name' in street:
                street['name']= name
                street['nums']= []
            elif street['name'] != name:
                json.dump(street,jsonCallejero)
                street = {}
                street['name']=name
                street['nums']= []
            
            """ Miramos si 'num' ya existe en nums para los kilómetros de las carreteras,
                porque viene el del sentido de entrada y de salida, solo guardamos el de entrada
            """
            if (len(street['nums'])==0) or (row['NUMERO'] != street['nums'][-1]['num']) :
                num = {'num':row['NUMERO'], "location":{'type':'Point',
                                                        'coordinates':[ conversion(row['LATITUD']),
                                                                        conversion(row['LONGITUD'])]}}
                street['nums'].append(num)
            
            
        json.dump(street,jsonCallejero)
        fCallejero.close()
        jsonCallejero.close()
        